In [1]:
instance_uri = "https://api.development.sinopia.io/resource/690eb696-98e6-4fa9-aecc-6f6ef774f339"
# change the instance_uri to the URI of the instance that you want to serialize for Alma RDF/XML.

In [2]:
!pip install rdflib
!pip install lxml


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from rdflib import Graph, Namespace, URIRef
from rdflib import Namespace, URIRef, RDF, Graph
from rdflib.namespace import RDF
from lxml import etree as ET
from name_space.alma_ns import alma_namespaces


In [4]:
instance_uri = URIRef(instance_uri)
work_uri = None
instance_graph = Graph()
work_graph = Graph()
instance_graph.parse(instance_uri)

<Graph identifier=N008a9de402424d0daeb44eb5c0dfe48e (<class 'rdflib.graph.Graph'>)>

In [5]:
# Define the bf and bflc namespaces
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")
for prefix, url in alma_namespaces:
    instance_graph.bind(prefix, url)
work_uri = instance_graph.value(subject=URIRef(instance_uri), predicate=bf.instanceOf)
work_uri = URIRef(work_uri)
# Explicitly state that work_uri is of type bf:Work
work_graph.add((work_uri, RDF.type, bf.Work))
# add the work to the instance graph
instance_graph.add((instance_uri, bf.instanceOf, URIRef(work_uri)))
# serialize the instance graph
instance_alma_xml = instance_graph.serialize(format="pretty-xml", encoding="utf-8")

print(instance_graph.serialize(format='pretty-xml'))


<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:bf="http://id.loc.gov/ontologies/bibframe/"
  xmlns:sinopiabf="http://sinopia.io/vocabulary/bf/"
  xmlns:sinopia="http://sinopia.io/vocabulary/"
  xmlns:bflc="http://id.loc.gov/ontologies/bflc/"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
>
  <bf:Instance rdf:about="https://api.development.sinopia.io/resource/690eb696-98e6-4fa9-aecc-6f6ef774f339">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Print"/>
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Instance</sinopia:hasResourceTemplate>
    <bf:instanceOf>
      <rdf:Description rdf:about="https://api.development.sinopia.io/resource/c61118ac-f5ad-4d1c-99c2-efb8c28898bf">
        <rdfs:label xml:lang="en">Johannes Gutenberg</rdfs:label>
      </rdf:Description>
    </bf:instanceOf>
    <bf:title>
      <bf:Title rdf:nodeID="fc0ee8765e6814e218e17400be7362059b1">
        <bf:mainTitle xml:lang="

In [6]:
tree = ET.fromstring(instance_alma_xml)
# apply xslt to normalize instance
xslt = ET.parse("xsl/normalize-instance-sinopia2alma.xsl")
transform = ET.XSLT(xslt)
instance_alma_xml = transform(tree)
instance_alma_xml = ET.tostring(
        instance_alma_xml, pretty_print=True, encoding="utf-8"
        )
# save the xml to a file
with open("alma-instance.xml", "wb") as f:
    f.write(instance_alma_xml)
print(instance_alma_xml.decode("utf-8"))

<bib>
  <record_format>lcbf_instance</record_format>
  <suppress_from_publishing>false</suppress_from_publishing>
  <record>
    <rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
  <bf:Instance rdf:about="https://api.development.sinopia.io/resource/690eb696-98e6-4fa9-aecc-6f6ef774f339" xmlns:bf="http://id.loc.gov/ontologies/bibframe/">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Print"/>
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Instance</sinopia:hasResourceTemplate>
    <bf:instanceOf rdf:resource="https://api.development.sinopia.io/resource/c61118ac-f5ad-4d1c-99c2-efb8c28898bf"/>
    <bf:title>
      <bf:Title rdf:nodeID="fc0ee8765e6814e218e17400be7362059b1">
        <bf:mainTitle xml:lang="de">Johannes Gutenberg</bf:mainTitle>
        <bflc:nonS